refs: 
* https://www.tensorflow.org/responsible_ai/model_card_toolkit/guide
* https://www.tensorflow.org/responsible_ai/model_card_toolkit/guide/concepts
* [Annotate Modelcard information](https://www.tensorflow.org/responsible_ai/model_card_toolkit/examples/Scikit_Learn_Model_Card_Toolkit_Demo#annotate_information_into_model_card)
* [Extending MC with RAI tests](https://github.com/cylynx/verifyml/blob/d3b4b3465493b802aba87edff81015f0db97805b/verifyml/model_card_toolkit/model_card.py#L333) 
* [Model Card Source](https://github.com/tensorflow/model-card-toolkit/blob/main/model_card_toolkit/core.py)

In [1]:
# add dev src path to system
import sys

# Specify the path of the directory containing the modules
dev_module_directory = '../frontend/src'

# Add the directory to the sys.path list
sys.path.append(dev_module_directory)

In [ ]:
import core_engine

In [ ]:
mo = core_engine.ModelOwner()

# Test psycopg

In [ ]:
import psycopg

In [ ]:
# this works in terminal but on ipykernel ... coould be asyncio issue
with psycopg.connect("host=127.0.0.1 port=5432 dbname=postgres user=postgres password=postgres") as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM owner")
        cur.fetchall()
        records = []
        for record in cur:
            print(record)


#will print 
#<psycopg.Cursor [TUPLES_OK] [INTRANS] (host=127.0.0.1 database=postgres) at 0xffffb2c64720>
#[('Quantum AI Sdn Bhd', 'sengtak.goh@cognigen.xyz', 1, UUID('4129bd65-3ea3-4de9-a5cc-6122106be3dd'))]

In [ ]:
records

## Test Loading Jsonb modelcard from Postgresql

In [2]:
from utils import psql_database_interface

In [7]:
configs = {
    "host":"127.0.0.1", 
    'port':5432,
    'dbname':'postgres',
    'user':'postgres',
    'password':'postgres'
}
qry_str = "SELECT name, version, generated_at, modelcard_data FROM modelcard WHERE email = 'sengtak@hotmail.com';"
action_str = "query"


r = psql_database_interface(qry_str, configs, action_str)

In [8]:
r

[('Test',
  'demo',
  datetime.datetime(2024, 2, 2, 11, 25, 36, 985119, tzinfo=zoneinfo.ZoneInfo(key='Etc/UTC')),
  {'model_details': {'name': 'Test',
    'owners': [{'name': 'Quantum AI Sdn Bhd',
      'contact': 'sengtak@hotmail.com'}],
    'version': {'name': 'demo'}},
   'schema_version': '0.0.2',
   'quantitative_analysis': {'graphics': {'collection': [{'name': 'Fairness Metrics',
       'image': '...'}]},
    'performance_metrics': [{'type': 'accuracy',
      'slice': 'Female',
      'value': '0.7630'},
     {'type': 'accuracy', 'slice': 'Male', 'value': '0.6244'},
     {'type': 'precision', 'slice': 'Female', 'value': '0.1140'},
     {'type': 'precision', 'slice': 'Male', 'value': '0.3009'},
     {'type': 'false positive rate', 'slice': 'Female', 'value': '0.1687'},
     {'type': 'false positive rate', 'slice': 'Male', 'value': '0.1633'},
     {'type': 'false negative rate', 'slice': 'Female', 'value': '0.8159'},
     {'type': 'false negative rate', 'slice': 'Male', 'value': '0.

In [10]:
mc_json = r[0][-1] #first record, last column

In [11]:
import model_card_toolkit as mctlib
mct = mctlib.ModelCardToolkit()
model_card = mct.scaffold_assets(mc_json) #use the json loaded from database to populate the modelcard

In [12]:
model_card

ModelCard(model_details=ModelDetails(name='Test', overview=None, documentation=None, owners=[Owner(name='Quantum AI Sdn Bhd', contact='sengtak@hotmail.com')], version=Version(name='demo', date=None, diff=None), licenses=[], references=[], citations=[], path=None), model_parameters=ModelParameters(model_architecture=None, data=[], input_format=None, input_format_map=[], output_format=None, output_format_map=[]), quantitative_analysis=QuantitativeAnalysis(performance_metrics=[PerformanceMetric(type='accuracy', value='0.7630', slice='Female', confidence_interval=ConfidenceInterval(lower_bound=None, upper_bound=None)), PerformanceMetric(type='accuracy', value='0.6244', slice='Male', confidence_interval=ConfidenceInterval(lower_bound=None, upper_bound=None)), PerformanceMetric(type='precision', value='0.1140', slice='Female', confidence_interval=ConfidenceInterval(lower_bound=None, upper_bound=None)), PerformanceMetric(type='precision', value='0.3009', slice='Male', confidence_interval=Conf

# generate list of countries

[turn list to enum](https://stackoverflow.com/questions/62120732/generate-an-enum-class-from-a-list-in-python)

In [ ]:
from utils import generate_enum
import pycountry

In [ ]:
def init_country_list (): 
  cntry = []
  for c in list(pycountry.countries):
    cntry += [(c.alpha_2, c.name)]
  return cntry


In [ ]:
CountryList = init_country_list()
print(generate_enum('EnumCountry',dict(CountryList)))

# Test UCI Dataset
https://fairlearn.org/v0.10/quickstart.html#loading-the-dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fairlearn.datasets import fetch_adult
data = fetch_adult(as_frame=True)
X = pd.get_dummies(data.data)
y_true = (data.target == '>50K') * 1
sex = data.data['sex']

In [ ]:
class SAMPLER:
    def __init__(self, X, y, s):
        self.X = X
        self.y = y
        self.s = s

    def __call__(self,idx):
        for i in idx:
            yield (self.X.loc[i], self.y.loc[i], self.s.loc[i])

In [ ]:
sampler = SAMPLER(X, y_true, sex)

In [ ]:
g = sampler([1,2])

In [ ]:
for d in g:
    print(d)

In [ ]:
X.iloc[:]

In [ ]:
len(X)

# Test FastAPI client

In [ ]:
import requests

In [ ]:
response = requests.get("http://127.0.0.1:8000/get_size")

In [ ]:
response.status_code

In [ ]:
int(response.content)

In [ ]:
# list
import json
idx = [0,1,2]
body = json.dumps(idx)
body

In [ ]:
body = json.dumps([i for i in range(32)]) # super inefficient if loads 48842 in 1 go, need batching

In [ ]:
# Set headers for sending the post request
headers = {"Content-Type": "application/json"}

# Send the POST request to your FastAPI endpoint.
response = requests.post("http://localhost:8000/sample", data=body, headers=headers)
response.status_code

In [ ]:
response.content

In [ ]:
import pandas as pd

In [ ]:
X_ = json.loads(response.content)["feature"]
X = pd.DataFrame(X_)
X.head()

In [ ]:
X_

In [ ]:
X.columns.size

In [ ]:
y_true_ = json.loads(response.content)["label"]
#y_true
y_true ={'label':y_true_}
y_true = pd.DataFrame(y_true)
y_true

In [ ]:
y_true_

In [ ]:
s_ = json.loads(response.content)["sensitive"]
s_

In [ ]:
X.to_dict() #flatten it

In [ ]:
response = requests.post("http://localhost:8000/predict", data=json.dumps(X.to_dict()), headers=headers)
response.status_code

In [ ]:
response.content

In [ ]:
y_pred = json.loads(response.content)
y_pred = pd.DataFrame(y_pred)
y_pred

In [ ]:
from fairlearn.datasets import fetch_adult

In [ ]:
data = fetch_adult(as_frame=True)
X = pd.get_dummies(data.data)
y_true = (data.target == '>50K') * 1
sex = data.data['sex']

In [ ]:
type(sex)

In [ ]:
type(y_true)

In [ ]:
type(X)

In [ ]:
yf = y_true.to_frame(name="label")

In [ ]:
type(yf["label"])

In [ ]:
import pandas as pd

In [ ]:
# Define your data
data = {
    'sex':['F','M'],
    'category1': [1, 2],
    'category2': [3, 4],
    'category3': [5, 6]
}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data)
df.set_index(['sex'], inplace=True)
df


In [ ]:
df.columns[0]

In [ ]:
df.index[0]

In [ ]:
df['category1']['F']

## Gen AI

### Gemini-pro 
https://ai.google.dev/tutorials/python_quickstart

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))